database.yml 형식에 맞게 구성되어있지 않을 경우
- 하단 실행
- 파일명 변경이니 주의 필요

In [ ]:
# import os

# # 오디오 파일 이름 맞추기기, 돌릴 때 조심/한번만 돌리기.
# for file in os.listdir('Train_Ali_far/audio_dir'):
#     file_name = file.split('.')[0]
#     new_file_name = file_name.split('_')[0] + "_" + file_name.split('_')[1] + ".wav"
#     os.rename('Train_Ali_far/audio_dir/' + file, 'Train_Ali_far/audio_dir/' + new_file_name)

In [ ]:
# rttm_dir  = "Train_Ali_far/rttm"
# train_dir = "Train_Ali_far/rttm/train/"
# test_dir = "Train_Ali_far/rttm/test/"
# dev_dir = "Train_Ali_far/rttm/dev/"

# os.makedirs(train_dir, exist_ok=True)
# os.makedirs(test_dir, exist_ok=True)
# os.makedirs(dev_dir, exist_ok=True)

# tg_files = [f for f in os.listdir(rttm_dir) if f.endswith('.rttm')]

# # train, test, dev split
# train_count = int(len(tg_files) * 0.8)
# test_count = int(len(tg_files) * 0.1)
# dev_count = len(tg_files) - train_count - test_count

# train_files = tg_files[:train_count]
# test_files = tg_files[train_count:train_count + test_count]
# dev_files = tg_files[train_count + test_count:]


# with open("Train_Ali_far/train_list.txt", "w", encoding="utf-8") as f:
#     for i, file in enumerate(train_files):
#         f.write(file.split('.')[0] + "\n")


# # test list txt file
# with open("Train_Ali_far/test_list.txt", "w", encoding="utf-8") as f:
#     for i, file in enumerate(test_files):
#         f.write(file.split('.')[0] + "\n")

# # dev list txt file
# with open("Train_Ali_far/dev_list.txt", "w", encoding="utf-8") as f:
#     for i, file in enumerate(dev_files):
#         f.write(file.split('.')[0] + "\n")

# # move files
# for f in train_files:
#     os.rename(os.path.join(rttm_dir, f), os.path.join(train_dir, f))

# for f in test_files:
#     os.rename(os.path.join(rttm_dir, f), os.path.join(test_dir, f))

# for f in dev_files:
#     os.rename(os.path.join(rttm_dir, f), os.path.join(dev_dir, f))
    

In [ ]:
# from pydub import AudioSegment
# import os

# os.mkdir('Train_Ali_far/uem')
# os.mkdir('Train_Ali_far/uem/train')
# os.mkdir('Train_Ali_far/uem/test')
# os.mkdir('Train_Ali_far/uem/dev')

# def get_duration(file):
#     audio = AudioSegment.from_wav(file)
#     return audio.duration_seconds

# #get train list file name
# train_list = []
# with open('Train_Ali_far/train_list.txt', 'r', encoding='utf-8') as f:
#     train_list = f.readlines()
# train_list = [x.strip() for x in train_list]

# #get test list file name
# test_list = []
# with open('Train_Ali_far/test_list.txt', 'r', encoding='utf-8') as f:
#     test_list = f.readlines()
# test_list = [x.strip() for x in test_list]

# #get dev list file name
# dev_list = []
# with open('Train_Ali_far/dev_list.txt', 'r', encoding='utf-8') as f:
#     dev_list = f.readlines()
# dev_list = [x.strip() for x in dev_list]


# for file in os.listdir('Train_Ali_far/audio_dir'):
#     file_name = file.split('.')[0]
#     duration = get_duration('Train_Ali_far/audio_dir/' + file) 

#     if file_name in train_list:
#         #generate uem file
#         with open('Train_Ali_far/uem/train/' + file_name + '.uem', 'w', encoding='utf-8') as f:
#             f.write(file_name + ' 1 ' + '0.00 ' + str(duration))
    
#     elif file_name in test_list:
#         #generate uem file
#         with open('Train_Ali_far/uem/test/' + file_name + '.uem', 'w', encoding='utf-8') as f:
#             f.write(file_name + ' 1 ' + '0.00 ' + str(duration))
    
#     elif file_name in dev_list:
#         #generate uem file
#         with open('Train_Ali_far/uem/dev/' + file_name + '.uem', 'w', encoding='utf-8') as f:
#             f.write(file_name + ' 1 ' + '0.00 ' + str(duration))

# 파인튜닝 

In [ ]:
from pyannote.database import registry, FileFinder

registry.load_database("/modu/docs/fine_tuning/database.yml")
dataset = registry.get_protocol("ALI.SpeakerDiarization.jojo", {"audio": FileFinder()})

In [ ]:
!PYANNOTE_DATABASE_CONFIG="database.yml" pyannote-database info ALI.SpeakerDiarization.jojo

In [ ]:
# Importing the base segmentation model for fine-tuning
from pyannote.audio import Pipeline
from dotenv import load_dotenv
import torch
import os

load_dotenv()
pretrained_pipeline = Pipeline.from_pretrained("pyannote/speaker-diarization-3.1", use_auth_token=os.getenv("HUGGINGFACE_TOKEN")) 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
pretrained_pipeline = pretrained_pipeline.to(device)

In [ ]:
from pyannote.metrics.diarization import DiarizationErrorRate

metric = DiarizationErrorRate()

for file in dataset.test():
    # apply pretrained pipeline
    file["pretrained pipeline"] = pretrained_pipeline(file)

    # evaluate its performance
    metric(file["annotation"], file["pretrained pipeline"], uem=file["annotated"])

print(f"The pretrained pipeline reaches a Diarization Error Rate (DER) of {100 * abs(metric):.1f}% on {dataset.name} test set.")

In [ ]:
file['annotation'] 

In [ ]:
file['pretrained pipeline']

# Fine-tuning the segmentation model

In [ ]:
from pyannote.audio import Model
import os
import torch
from dotenv import load_dotenv

load_dotenv()
model = Model.from_pretrained("pyannote/segmentation-3.0", use_auth_token=os.getenv("HUGGINGFACE_TOKEN"))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [ ]:
#####
from pyannote.audio import Model
import os
import torch

model = Model.from_pretrained("epoch=0.ckpt")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [ ]:
from pyannote.audio.tasks import Segmentation
task = Segmentation(
    dataset, 
    duration=model.specifications.duration, 
    max_num_speakers=len(model.specifications.classes), 
    batch_size=64,
    num_workers=4, 
    loss="bce", 
    vad_loss="bce")
model.task = task
model.prepare_data()
model.setup()

In [ ]:
# this takes approximately 15min to run on Google Colab GPU
from types import MethodType
from torch.optim import Adam
from pytorch_lightning.callbacks import (
    EarlyStopping,
    ModelCheckpoint,
    RichProgressBar,
)

# we use Adam optimizer with 1e-4 learning rate
def configure_optimizers(self):
    return Adam(self.parameters(), lr=1e-4)

model.configure_optimizers = MethodType(configure_optimizers, model)

# we monitor diarization error rate on the validation set
# and use to keep the best checkpoint and stop early
monitor, direction = task.val_monitor
checkpoint = ModelCheckpoint(
    monitor=monitor,
    mode=direction,
    save_top_k=1,
    every_n_epochs=1,
    save_last=False,
    save_weights_only=False,
    filename="{epoch}",
    verbose=False,
)
early_stopping = EarlyStopping(
    monitor=monitor,
    mode=direction,
    min_delta=0.0,
    patience=10,
    strict=True,
    verbose=False,
)

callbacks = [RichProgressBar(), checkpoint, early_stopping]

# we train for at most 20 epochs (might be shorter in case of early stopping)
from pytorch_lightning import Trainer
trainer = Trainer(accelerator="gpu", 
                  callbacks=callbacks, 
                  max_epochs=20,
                  gradient_clip_val=0.5)
trainer.fit(model)

In [ ]:
finetuned_model = checkpoint.best_model_path

#to file
import shutil
shutil.move(finetuned_model, "finetuned_model.ckpt")


In [ ]:
# 체크포인트 로딩 시 실행.
# we load the best model
from pyannote.audio import Model
import torch

finetuned_model = "epoch=0.ckpt"
model = Model.from_pretrained(finetuned_model)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

### Optimizing the pipeline hyper parameters

In [ ]:
pretrained_hyperparameters = pretrained_pipeline.parameters(instantiated=True)
pretrained_hyperparameters

In [ ]:
# this takes approximately 5min to run on Google Colab GPU
from pyannote.audio.pipelines import SpeakerDiarization
from pyannote.pipeline import Optimizer

pipeline = SpeakerDiarization(
    segmentation=finetuned_model,
    clustering="OracleClustering",
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
pipeline = pipeline.to(device)

# as reported in the technical report, min_duration_off can safely be set to 0.0
pipeline.freeze({"segmentation": {"min_duration_off": 0.0}})

optimizer = Optimizer(pipeline)
dev_set = list(dataset.development())

iterations = optimizer.tune_iter(dev_set, show_progress=True)
best_loss = 1.0
for i, iteration in enumerate(iterations):
    print(f"Iteration {i}")
    print(f"Current segmentation threshold: {iteration['params']['segmentation']['threshold']}")
    print(f"Current segmentation params: {iteration['params']['segmentation']}")  # segmentation 파라미터 출력
    if i > 20: break  # 50 iterations should give slightly better results

In [ ]:
best_segmentation_threshold = optimizer.best_params["segmentation"]["threshold"]

In [ ]:

# this takes approximately 5min to run on Google Colab GPU
pipeline = SpeakerDiarization(
    segmentation=finetuned_model,
    embedding=pretrained_pipeline.embedding,
    embedding_exclude_overlap=pretrained_pipeline.embedding_exclude_overlap,
    clustering=pretrained_pipeline.klustering,
)

pipeline.freeze({
    "segmentation": {
        "threshold": best_segmentation_threshold,
        "min_duration_off": 0.0,
    },
    "clustering": {
        "method": "centroid",
        "min_cluster_size": 15,
    },
})

optimizer = Optimizer(pipeline)
iterations = optimizer.tune_iter(dev_set, show_progress=False)
best_loss = 1.0
for i, iteration in enumerate(iterations):
    print(f"Best clustering threshold so far: {iteration['params']['clustering']['threshold']}")
    if i > 20: break  # 50 iterations should give slightly better results

In [ ]:
best_clustering_threshold = optimizer.best_params['clustering']['threshold']

In [ ]:
# this takes approximately 2min to run on Google Colab GPU
finetuned_pipeline = SpeakerDiarization(
    segmentation=finetuned_model,
    embedding=pretrained_pipeline.embedding,
    embedding_exclude_overlap=pretrained_pipeline.embedding_exclude_overlap,
    clustering=pretrained_pipeline.klustering,
)

finetuned_pipeline.instantiate({
    "segmentation": {
        "threshold": best_segmentation_threshold,
        "min_duration_off": 0.0,
    },
    "clustering": {
        "method": "centroid",
        "min_cluster_size": 15,
        "threshold": best_clustering_threshold,
    },
})

metric = DiarizationErrorRate()

for file in dataset.test():
    # apply finetuned pipeline
    file["finetuned pipeline"] = finetuned_pipeline(file)

    # evaluate its performance
    metric(file["annotation"], file["finetuned pipeline"], uem=file["annotated"])

print(f"The finetuned pipeline reaches a Diarization Error Rate (DER) of {100 * abs(metric):.1f}% on {dataset.name} test set.")